In [1]:
# Imports
import pandas as pd
import numpy as np
import math
import datetime
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Load the data
games_df = pd.read_csv('/Users/JKMacBook/Documents/Lambda/Kaggle/NFL/nfl-big-data-bowl-2021/games.csv')
players_df = pd.read_csv('/Users/JKMacBook/Documents/Lambda/Kaggle/NFL/nfl-big-data-bowl-2021/players.csv')
plays_df = pd.read_csv('/Users/JKMacBook/Documents/Lambda/Kaggle/NFL/nfl-big-data-bowl-2021/full_play_2.csv')
week1r_df = pd.read_csv('/Users/JKMacBook/Documents/Lambda/Kaggle/NFL/nfl-big-data-bowl-2021/week1.csv')
week2r_df = pd.read_csv('/Users/JKMacBook/Documents/Lambda/Kaggle/NFL/nfl-big-data-bowl-2021/week2.csv')
week3r_df = pd.read_csv('/Users/JKMacBook/Documents/Lambda/Kaggle/NFL/nfl-big-data-bowl-2021/week3.csv')
week4r_df = pd.read_csv('/Users/JKMacBook/Documents/Lambda/Kaggle/NFL/nfl-big-data-bowl-2021/week4.csv')
week5r_df = pd.read_csv('/Users/JKMacBook/Documents/Lambda/Kaggle/NFL/nfl-big-data-bowl-2021/week5.csv')
week6r_df = pd.read_csv('/Users/JKMacBook/Documents/Lambda/Kaggle/NFL/nfl-big-data-bowl-2021/week6.csv')
week7r_df = pd.read_csv('/Users/JKMacBook/Documents/Lambda/Kaggle/NFL/nfl-big-data-bowl-2021/week7.csv')
week8r_df = pd.read_csv('/Users/JKMacBook/Documents/Lambda/Kaggle/NFL/nfl-big-data-bowl-2021/week8.csv')
week9r_df = pd.read_csv('/Users/JKMacBook/Documents/Lambda/Kaggle/NFL/nfl-big-data-bowl-2021/week9.csv')
week10r_df = pd.read_csv('/Users/JKMacBook/Documents/Lambda/Kaggle/NFL/nfl-big-data-bowl-2021/week10.csv')
week11r_df = pd.read_csv('/Users/JKMacBook/Documents/Lambda/Kaggle/NFL/nfl-big-data-bowl-2021/week11.csv')
week12r_df = pd.read_csv('/Users/JKMacBook/Documents/Lambda/Kaggle/NFL/nfl-big-data-bowl-2021/week12.csv')
week13r_df = pd.read_csv('/Users/JKMacBook/Documents/Lambda/Kaggle/NFL/nfl-big-data-bowl-2021/week13.csv')
week14r_df = pd.read_csv('/Users/JKMacBook/Documents/Lambda/Kaggle/NFL/nfl-big-data-bowl-2021/week14.csv')
week15r_df = pd.read_csv('/Users/JKMacBook/Documents/Lambda/Kaggle/NFL/nfl-big-data-bowl-2021/week15.csv')
week16r_df = pd.read_csv('/Users/JKMacBook/Documents/Lambda/Kaggle/NFL/nfl-big-data-bowl-2021/week16.csv')
week17r_df = pd.read_csv('/Users/JKMacBook/Documents/Lambda/Kaggle/NFL/nfl-big-data-bowl-2021/week17.csv')

In [3]:
# The plays df
plays_df.head()

,Unnamed: 0,Unnamed: 0.1,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,playType,...,isDefensivePI,gameDate,gameTimeEastern,homeTeamAbbr,visitorTeamAbbr,week,def,defender,manZone,scheme
0,0,0,2018090600,75,(15:00) M.Ryan pass short right to J.Jones pus...,1,1,15,ATL,play_type_pass,...,False,09/06/2018,20:20:00,PHI,ATL,1,PHI,Malcolm Jenkins,zone,cover-3
1,1,1,2018090600,146,(13:10) M.Ryan pass incomplete short right to ...,1,1,10,ATL,play_type_pass,...,False,09/06/2018,20:20:00,PHI,ATL,1,PHI,Jordan Hicks,zone,cover-3
2,2,2,2018090600,168,(13:05) (Shotgun) M.Ryan pass incomplete short...,1,2,10,ATL,play_type_pass,...,False,09/06/2018,20:20:00,PHI,ATL,1,PHI,Sidney Jones,man,cover-1
3,3,3,2018090600,190,(13:01) (Shotgun) M.Ryan pass deep left to J.J...,1,3,10,ATL,play_type_pass,...,False,09/06/2018,20:20:00,PHI,ATL,1,PHI,Ronald Darby,man,cover-1
4,4,4,2018090600,256,(10:59) (Shotgun) M.Ryan pass incomplete short...,1,3,1,ATL,play_type_pass,...,False,09/06/2018,20:20:00,PHI,ATL,1,PHI,Malcolm Jenkins,man,cover-0


In [4]:
# Reduce the weeks data to pass_arrived data
def week_reduce(df):
    df = df[(df['event'] == 'pass_arrived')]
    return df

week1_df = week_reduce(week1r_df)
week2_df = week_reduce(week2r_df)
week3_df = week_reduce(week3r_df)
week4_df = week_reduce(week4r_df)
week5_df = week_reduce(week5r_df)
week6_df = week_reduce(week6r_df)
week7_df = week_reduce(week7r_df)
week8_df = week_reduce(week8r_df)
week9_df = week_reduce(week9r_df)
week10_df = week_reduce(week10r_df)
week11_df = week_reduce(week11r_df)
week12_df = week_reduce(week12r_df)
week13_df = week_reduce(week13r_df)
week14_df = week_reduce(week14r_df)
week15_df = week_reduce(week15r_df)
week16_df = week_reduce(week16r_df)
week17_df = week_reduce(week17r_df)

In [5]:
# Create a new column identifying offensive and defensive players
def weekly_conv(df):
    df['defPlayer'] = np.where((df['position'] == 'SS') | 
                                 (df['position'] == 'FS') |
                                 (df['position'] == 'MLB') | 
                                 (df['position'] == 'CB') |
                                 (df['position'] == 'LB') | 
                                 (df['position'] == 'OLB') |
                                 (df['position'] == 'ILB') | 
                                 (df['position'] == 'DL') |
                                 (df['position'] == 'DB') | 
                                 (df['position'] == 'NT') |
                                 (df['position'] == 'S') | 
                                 (df['position'] == 'DE') |
                                 (df['position'] == 'DT'),
                                 'D', 'O')
    
    return df.sort_values(by=['gameId', 'playId', 'frameId', 'team'])

w1_df = weekly_conv(week1_df)
w2_df = weekly_conv(week2_df)
w3_df = weekly_conv(week3_df)
w4_df = weekly_conv(week4_df)
w5_df = weekly_conv(week5_df)
w6_df = weekly_conv(week6_df)
w7_df = weekly_conv(week7_df)
w8_df = weekly_conv(week8_df)
w9_df = weekly_conv(week9_df)
w10_df = weekly_conv(week10_df)
w11_df = weekly_conv(week11_df)
w12_df = weekly_conv(week12_df)
w13_df = weekly_conv(week13_df)
w14_df = weekly_conv(week14_df)
w15_df = weekly_conv(week15_df)
w16_df = weekly_conv(week16_df)
w17_df = weekly_conv(week17_df)

In [6]:
# The week1 data
w1_df.head(20)

,time,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,frameId,team,gameId,playId,playDirection,route,defPlayer
644,2018-09-07T01:07:19.200Z,96.30,27.52,0.59,0.51,0.06,333.07,1.23,pass_arrived,310.0,Matt Ryan,2.0,QB,47,away,2018090600,75,left,NaN,O
646,2018-09-07T01:07:19.200Z,81.11,47.87,4.23,1.63,0.42,53.31,22.58,pass_arrived,2495454.0,Julio Jones,11.0,WR,47,away,2018090600,75,left,HITCH,O
648,2018-09-07T01:07:19.200Z,77.66,35.52,2.99,3.53,0.31,78.88,305.48,pass_arrived,2533040.0,Mohamed Sanu,12.0,WR,47,away,2018090600,75,left,HITCH,O
650,2018-09-07T01:07:19.200Z,91.13,30.53,3.85,2.91,0.38,246.65,294.36,pass_arrived,2543583.0,Devonta Freeman,24.0,RB,47,away,2018090600,75,left,NaN,O
654,2018-09-07T01:07:19.200Z,78.35,16.74,7.05,4.26,0.70,150.73,197.08,pass_arrived,2555415.0,Austin Hooper,81.0,TE,47,away,2018090600,75,left,OUT,O
656,2018-09-07T01:07:19.200Z,87.13,10.93,6.81,2.09,0.69,86.71,193.39,pass_arrived,2559033.0,Ricky Ortiz,30.0,FB,47,away,2018090600,75,left,FLAT,O
657,2018-09-07T01:07:19.200Z,83.29,46.47,12.68,16.51,1.90,NaN,NaN,pass_arrived,NaN,Football,NaN,NaN,47,football,2018090600,75,left,NaN,O
645,2018-09-07T01:07:19.200Z,82.34,45.18,7.57,1.72,0.75,354.68,353.52,pass_arrived,79848.0,Malcolm Jenkins,27.0,SS,47,home,2018090600,75,left,NaN,D
647,2018-09-07T01:07:19.200Z,82.90,17.62,3.64,4.01,0.36,13.28,299.37,pass_arrived,2495613.0,Corey Graham,24.0,FS,47,home,2018090600,75,left,NaN,D
649,2018-09-07T01:07:19.200Z,70.28,35.09,6.10,2.74,0.59,32.94,29.19,pass_arrived,2534832.0,Rodney McLeod,23.0,FS,47,home,2018090600,75,left,NaN,D


In [7]:
# Set the column names for the result df
column_names = ['weekId', 'gameId', 'playId', 'oPlayer', 'dPlayer', 'route', 'endDist', 'endTime']
result_df = pd.DataFrame(columns = column_names)

In [8]:
# Create a result df containing the closest defender to each receiver running a route
def get_closest_dplayer(wid, gid, pid, eid, r_df):
    
    print(wid, gid, pid, eid)
    
    if wid == 1:
        w_df = w1_df
    elif wid == 2:
        w_df = w2_df
    elif wid == 3:
        w_df = w3_df
    elif wid == 4:
        w_df = w4_df
    elif wid == 5:
        w_df = w5_df
    elif wid == 6:
        w_df = w6_df
    elif wid == 7:
        w_df = w7_df
    elif wid == 8:
        w_df = w8_df
    elif wid == 9:
        w_df = w9_df
    elif wid == 10:
        w_df = w10_df
    elif wid == 11:
        w_df = w11_df
    elif wid == 12:
        w_df = w12_df
    elif wid == 13:
        w_df = w13_df
    elif wid == 14:
        w_df = w14_df
    elif wid == 15:
        w_df = w15_df
    elif wid == 16:
        w_df = w16_df
    elif wid == 17:
        w_df = w17_df
    
    # Create a frame df
    f_df = w_df.loc[(w_df['playId'] == pid) & (w_df['gameId'] == gid) & (w_df['event'] == eid)]
    
    # Create a df for defenders (d_df) and receivers (o_df)
    d_df = f_df.loc[(f_df['defPlayer'] == 'D') & 
                    (f_df['displayName'] != 'Football')]
    o_df = f_df.loc[(f_df['defPlayer'] == 'O') & 
                    (f_df['displayName'] != 'Football') & 
                    (f_df['position'] != 'QB') & 
                    (f_df['route'].notnull())]
    
    # Test for empty df's
    if d_df.empty | o_df.empty:
        return r_df

    # For each offensive reciever find closest defender
    for i in o_df.index:
        
        # Offensive player location, name and route
        opos_x = o_df.at[i, 'x']
        opos_y = o_df.at[i, 'y']
        o_player = o_df.at[i,'displayName']
        route = o_df.at[i, 'route']
        
        # Calculate the distance
        for i in d_df.index:
            d_df.at[i, 'bd'] = math.sqrt(((opos_x - d_df.at[i, 'x'])**2) + 
                                         ((opos_y - d_df.at[i, 'y'])**2))
        
        d_df['rank'] = d_df['bd'].rank()
        if d_df.loc[d_df['rank'] == 1]['displayName'].empty:
            dist = 100
        else:
            d_player = d_df.loc[d_df['rank'] == 1]['displayName'].item()
            
            p_time = d_df.loc[d_df['rank'] == 1]['time'].item()
            dt_string = p_time[11:23]
            dt_obj = datetime.datetime.strptime(dt_string, '%H:%M:%S.%f')
            
            dist = d_df.loc[d_df['rank'] == 1]['bd'].item()
            row = {'weekId': wid, 'gameId': gid, 'playId': pid, 'oPlayer': o_player, 
                   'dPlayer': d_player, 'route': route, 'endDist': dist, 'endTime': dt_obj}
            r_df = r_df.append(row, ignore_index = True)
            d_df.drop(d_df[d_df['displayName'] == d_player].index, inplace = True)
            
    return r_df

# column_names = ['weekId', 'gameId', 'playId', 'oPlayer', 'dPlayer', 'route', endDist', 'endTime']
# test_df = pd.DataFrame(columns = column_names)
# test_df = get_closest_dplayer(2, 2018091604, 2568, 'pass_arrived', test_df)
# test_df

In [9]:
for i in plays_df.index:
    
    result_df = get_closest_dplayer(plays_df.at[i, 'week'], 
                                    plays_df.at[i, 'gameId'],
                                    plays_df.at[i, 'playId'],
                                    'pass_arrived',
                                    result_df)

result_df.head(20) 

1 2018090600 75 pass_arrived
1 2018090600 146 pass_arrived
1 2018090600 168 pass_arrived
1 2018090600 190 pass_arrived
1 2018090600 256 pass_arrived
1 2018090600 320 pass_arrived
1 2018090600 344 pass_arrived
1 2018090600 402 pass_arrived
1 2018090600 492 pass_arrived
1 2018090600 521 pass_arrived
1 2018090600 545 pass_arrived
1 2018090600 636 pass_arrived
1 2018090600 752 pass_arrived
1 2018090600 776 pass_arrived
1 2018090600 839 pass_arrived
1 2018090600 889 pass_arrived
1 2018090600 949 pass_arrived
1 2018090600 992 pass_arrived
1 2018090600 1037 pass_arrived
1 2018090600 1061 pass_arrived
1 2018090600 1085 pass_arrived
1 2018090600 1202 pass_arrived
1 2018090600 1295 pass_arrived
1 2018090600 1344 pass_arrived
1 2018090600 1423 pass_arrived
1 2018090600 1473 pass_arrived
1 2018090600 1546 pass_arrived
1 2018090600 1568 pass_arrived
1 2018090600 1628 pass_arrived
1 2018090600 1680 pass_arrived
1 2018090600 1750 pass_arrived
1 2018090600 1779 pass_arrived
1 2018090600 1801 pass_arri

1 2018090902 1697 pass_arrived
1 2018090902 1969 pass_arrived
1 2018090902 3031 pass_arrived
1 2018090902 3233 pass_arrived
1 2018090903 79 pass_arrived
1 2018090903 149 pass_arrived
1 2018090903 194 pass_arrived
1 2018090903 218 pass_arrived
1 2018090903 333 pass_arrived
1 2018090903 382 pass_arrived
1 2018090903 424 pass_arrived
1 2018090903 451 pass_arrived
1 2018090903 513 pass_arrived
1 2018090903 596 pass_arrived
1 2018090903 641 pass_arrived
1 2018090903 663 pass_arrived
1 2018090903 685 pass_arrived
1 2018090903 762 pass_arrived
1 2018090903 791 pass_arrived
1 2018090903 813 pass_arrived
1 2018090903 837 pass_arrived
1 2018090903 878 pass_arrived
1 2018090903 941 pass_arrived
1 2018090903 1074 pass_arrived
1 2018090903 1146 pass_arrived
1 2018090903 1235 pass_arrived
1 2018090903 1336 pass_arrived
1 2018090903 1444 pass_arrived
1 2018090903 1515 pass_arrived
1 2018090903 1556 pass_arrived
1 2018090903 1585 pass_arrived
1 2018090903 1947 pass_arrived
1 2018090903 1971 pass_arriv

1 2018090907 2396 pass_arrived
1 2018090907 2418 pass_arrived
1 2018090907 2463 pass_arrived
1 2018090907 2511 pass_arrived
1 2018090907 2556 pass_arrived
1 2018090907 2618 pass_arrived
1 2018090907 2642 pass_arrived
1 2018090907 2664 pass_arrived
1 2018090907 2686 pass_arrived
1 2018090907 2744 pass_arrived
1 2018090907 2787 pass_arrived
1 2018090907 2853 pass_arrived
1 2018090907 2895 pass_arrived
1 2018090907 2957 pass_arrived
1 2018090907 3033 pass_arrived
1 2018090907 3055 pass_arrived
1 2018090907 3077 pass_arrived
1 2018090907 3122 pass_arrived
1 2018090907 3171 pass_arrived
1 2018090907 3193 pass_arrived
1 2018090907 3259 pass_arrived
1 2018090907 3332 pass_arrived
1 2018090907 3430 pass_arrived
1 2018090907 3475 pass_arrived
1 2018090907 3522 pass_arrived
1 2018090907 3567 pass_arrived
1 2018090907 3692 pass_arrived
1 2018090907 3716 pass_arrived
1 2018090907 3755 pass_arrived
1 2018090907 3777 pass_arrived
1 2018090907 3826 pass_arrived
1 2018090907 3871 pass_arrived
1 201809

1 2018090912 732 pass_arrived
1 2018090912 756 pass_arrived
1 2018090912 795 pass_arrived
1 2018090912 919 pass_arrived
1 2018090912 983 pass_arrived
1 2018090912 1081 pass_arrived
1 2018090912 1103 pass_arrived
1 2018090912 1127 pass_arrived
1 2018090912 1216 pass_arrived
1 2018090912 1259 pass_arrived
1 2018090912 1300 pass_arrived
1 2018090912 1371 pass_arrived
1 2018090912 1395 pass_arrived
1 2018090912 1419 pass_arrived
1 2018090912 1464 pass_arrived
1 2018090912 1493 pass_arrived
1 2018090912 1555 pass_arrived
1 2018090912 1617 pass_arrived
1 2018090912 1639 pass_arrived
1 2018090912 1666 pass_arrived
1 2018090912 1702 pass_arrived
1 2018090912 1724 pass_arrived
1 2018090912 1837 pass_arrived
1 2018090912 1882 pass_arrived
1 2018090912 1939 pass_arrived
1 2018090912 1963 pass_arrived
1 2018090912 1985 pass_arrived
1 2018090912 2030 pass_arrived
1 2018090912 2080 pass_arrived
1 2018090912 2115 pass_arrived
1 2018090912 2205 pass_arrived
1 2018090912 2258 pass_arrived
1 2018090912 

2 2018091300 2506 pass_arrived
2 2018091300 2547 pass_arrived
2 2018091300 2588 pass_arrived
2 2018091300 2705 pass_arrived
2 2018091300 2727 pass_arrived
2 2018091300 2826 pass_arrived
2 2018091300 2850 pass_arrived
2 2018091300 2908 pass_arrived
2 2018091300 2934 pass_arrived
2 2018091300 2979 pass_arrived
2 2018091300 3024 pass_arrived
2 2018091300 3055 pass_arrived
2 2018091300 3147 pass_arrived
2 2018091300 3192 pass_arrived
2 2018091300 3239 pass_arrived
2 2018091300 3301 pass_arrived
2 2018091300 3351 pass_arrived
2 2018091300 3375 pass_arrived
2 2018091300 3397 pass_arrived
2 2018091300 3421 pass_arrived
2 2018091300 3487 pass_arrived
2 2018091300 3530 pass_arrived
2 2018091300 3571 pass_arrived
2 2018091300 3595 pass_arrived
2 2018091300 3640 pass_arrived
2 2018091300 3685 pass_arrived
2 2018091300 3722 pass_arrived
2 2018091300 3746 pass_arrived
2 2018091300 3770 pass_arrived
2 2018091300 3832 pass_arrived
2 2018091300 3873 pass_arrived
2 2018091300 3895 pass_arrived
2 201809

...

17 2018123003 3335 pass_arrived
17 2018123003 3357 pass_arrived
17 2018123003 3403 pass_arrived
17 2018123003 3468 pass_arrived
17 2018123003 3532 pass_arrived
17 2018123003 3569 pass_arrived
17 2018123003 3591 pass_arrived
17 2018123003 3616 pass_arrived
17 2018123003 3638 pass_arrived
17 2018123003 3685 pass_arrived
17 2018123003 3707 pass_arrived
17 2018123003 3735 pass_arrived
17 2018123003 3804 pass_arrived
17 2018123003 3875 pass_arrived
17 2018123003 3906 pass_arrived
17 2018123003 4125 pass_arrived
17 2018123004 78 pass_arrived
17 2018123004 110 pass_arrived
17 2018123004 132 pass_arrived
17 2018123004 201 pass_arrived
17 2018123004 290 pass_arrived
17 2018123004 321 pass_arrived
17 2018123004 385 pass_arrived
17 2018123004 413 pass_arrived
17 2018123004 442 pass_arrived
17 2018123004 495 pass_arrived
17 2018123004 542 pass_arrived
17 2018123004 589 pass_arrived
17 2018123004 611 pass_arrived
17 2018123004 675 pass_arrived
17 2018123004 697 pass_arrived
17 2018123004 810 pass_a

17 2018123008 735 pass_arrived
17 2018123008 757 pass_arrived
17 2018123008 777 pass_arrived
17 2018123008 819 pass_arrived
17 2018123008 929 pass_arrived
17 2018123008 971 pass_arrived
17 2018123008 993 pass_arrived
17 2018123008 1026 pass_arrived
17 2018123008 1101 pass_arrived
17 2018123008 1131 pass_arrived
17 2018123008 1206 pass_arrived
17 2018123008 1253 pass_arrived
17 2018123008 1369 pass_arrived
17 2018123008 1391 pass_arrived
17 2018123008 1474 pass_arrived
17 2018123008 1496 pass_arrived
17 2018123008 1595 pass_arrived
17 2018123008 1620 pass_arrived
17 2018123008 1667 pass_arrived
17 2018123008 1687 pass_arrived
17 2018123008 1784 pass_arrived
17 2018123008 1806 pass_arrived
17 2018123008 1877 pass_arrived
17 2018123008 1902 pass_arrived
17 2018123008 1946 pass_arrived
17 2018123008 1988 pass_arrived
17 2018123008 2030 pass_arrived
17 2018123008 2094 pass_arrived
17 2018123008 2116 pass_arrived
17 2018123008 2138 pass_arrived
17 2018123008 2271 pass_arrived
17 2018123008 2

17 2018123011 1388 pass_arrived
17 2018123011 1413 pass_arrived
17 2018123011 1509 pass_arrived
17 2018123011 1531 pass_arrived
17 2018123011 1573 pass_arrived
17 2018123011 1642 pass_arrived
17 2018123011 1734 pass_arrived
17 2018123011 1759 pass_arrived
17 2018123011 1784 pass_arrived
17 2018123011 1806 pass_arrived
17 2018123011 1828 pass_arrived
17 2018123011 1884 pass_arrived
17 2018123011 1926 pass_arrived
17 2018123011 1974 pass_arrived
17 2018123011 2016 pass_arrived
17 2018123011 2058 pass_arrived
17 2018123011 2109 pass_arrived
17 2018123011 2227 pass_arrived
17 2018123011 2416 pass_arrived
17 2018123011 2441 pass_arrived
17 2018123011 2577 pass_arrived
17 2018123011 2727 pass_arrived
17 2018123011 2812 pass_arrived
17 2018123011 2959 pass_arrived
17 2018123011 3009 pass_arrived
17 2018123011 3122 pass_arrived
17 2018123011 3196 pass_arrived
17 2018123011 3323 pass_arrived
17 2018123011 3348 pass_arrived
17 2018123011 3387 pass_arrived
17 2018123011 3459 pass_arrived
17 20181

17 2018123015 322 pass_arrived
17 2018123015 349 pass_arrived
17 2018123015 393 pass_arrived
17 2018123015 418 pass_arrived
17 2018123015 462 pass_arrived
17 2018123015 572 pass_arrived
17 2018123015 640 pass_arrived
17 2018123015 662 pass_arrived
17 2018123015 687 pass_arrived
17 2018123015 709 pass_arrived
17 2018123015 804 pass_arrived
17 2018123015 848 pass_arrived
17 2018123015 949 pass_arrived
17 2018123015 991 pass_arrived
17 2018123015 1044 pass_arrived
17 2018123015 1135 pass_arrived
17 2018123015 1160 pass_arrived
17 2018123015 1275 pass_arrived
17 2018123015 1300 pass_arrived
17 2018123015 1322 pass_arrived
17 2018123015 1375 pass_arrived
17 2018123015 1485 pass_arrived
17 2018123015 1585 pass_arrived
17 2018123015 1694 pass_arrived
17 2018123015 1768 pass_arrived
17 2018123015 1810 pass_arrived
17 2018123015 1874 pass_arrived
17 2018123015 1896 pass_arrived
17 2018123015 1921 pass_arrived
17 2018123015 1984 pass_arrived
17 2018123015 2009 pass_arrived
17 2018123015 2031 pas

,weekId,gameId,playId,oPlayer,dPlayer,route,endDist,endTime
0,1,2018090600,75,Julio Jones,Malcolm Jenkins,HITCH,2.957871,1900-01-01 01:07:19.200
1,1,2018090600,75,Mohamed Sanu,Nate Gerry,HITCH,5.000090,1900-01-01 01:07:19.200
2,1,2018090600,75,Austin Hooper,Ronald Darby,OUT,1.360588,1900-01-01 01:07:19.200
3,1,2018090600,75,Ricky Ortiz,Corey Graham,FLAT,7.915112,1900-01-01 01:07:19.200
4,1,2018090600,146,Devonta Freeman,Sidney Jones,FLAT,10.383549,1900-01-01 01:09:13.200
5,1,2018090600,146,Justin Hardy,Kamu Grugier-Hill,CROSS,3.198203,1900-01-01 01:09:13.200
6,1,2018090600,146,Austin Hooper,Malcolm Jenkins,FLAT,1.646147,1900-01-01 01:09:13.200
7,1,2018090600,146,Marvin Hall,Ronald Darby,HITCH,3.465776,1900-01-01 01:09:13.200
8,1,2018090600,146,Calvin Ridley,Jordan Hicks,HITCH,1.891719,1900-01-01 01:09:13.200
9,1,2018090600,190,Julio Jones,Ronald Darby,CORNER,3.898949,1900-01-01 01:10:40.099


In [10]:
result_df.to_csv('/Users/JKMacBook/Documents/Lambda/Kaggle/NFL/nfl-big-data-bowl-2021/distance.csv')